# Analyse av Trondheim Forecast

#### Import av nødvendige moduler og klasser:

In [ ]:
import pandas as pd
import numpy as np
from src.data_analysis import DataAnalyse
from src.data_reader import DataLeser

: 

#### Last inn renset data

In [ ]:
dataleser= DataLeser()
df_renset = dataleser.les_csv("data/renset_trondheim_forecast.csv")

#### Initialiser analyseobjekt:

In [ ]:
analyse = DataAnalyse(df_renset)

#### Statistisk analyse:
- Bruker pandas sine .mean(), .median() og .std() til å beregne de ønskede statistiske målene. 

In [ ]:
stat_summary = analyse.beskriv_data
print(stat_summary)

Hvorfor bruke gjennomsnitt, median og standardavvik? 
- Gjennomsnitt gir et mål på sentraltrender, men kan påvirkes av ekstremverdier som ofte oppstår i naturen.
- Median virker bra mot outliers og gir et stabilt bilde ved skjevfordelte data.
- Standardavvik sier noe om hvor mye verdiene i datasettet varierer rundt snittet.

#### Korrelasjon mellom Temperatur og Fuktighet:
- Viser sammenheng mellom to verdier i datasettet ved en korrelasjonskoeffisient (Pearson) som sier noe om hvor sterkt to variabler henger sammen. 
    - Positiv: temperatur stiger -> øker fuktighet
    - Negativ: temperatur stiger -> synker fuktighet
    - Nær null: liten eller ingen lineær sammenheng.

In [ ]:
kol1="Temperatur"
kol2="Fuktighet"
try:
    korrelasjon = analyse.korrelasjon(kol1, kol2)
    print(f"Korrelasjon mellom {kol1} og {kol2}: {korrelasjon:.3f}")
except ValueError as e:
    print(e)

#### Indetifiser Outliers:
- Outliers kan oppstå på grunn av feil i måling eller ekstreme hendelser.
- I analysen av vær så vil outliers ofte oppstå grunnet endringer i vær. vi kan identifisere dem og senere inkludere dem i visualiseringen og prediktive modelleringer for å se hvordan de påvirker de ikke uteliggende verdiene.

In [ ]:
outliers = analyse.identifiser_outliers()
print(f"Antall rader der outliers identifiseres: {len(outliers)}")

- Metoden finner outliers ved å beregne en score:
$$
\text{score} = \frac{{observasjonsverdi} - \text{gjennomsnitt}}{\text{standardavvik}}
$$
- En høy verdi for score impliserer at det er en outlier.
- Metoden bestemmer terskelverdien på 3.0 som er vanlig praksis innen statistikk.

#### Trendanalyse for temperatur:
- Trendanalyse med glidende gjenomsnitt

Ved å bruke rullerende gjennomsnitt over for eksempel 7 dager, glatter vi ut tilfeldige svingiger. Dette gir bedre innsikt for:
- Klimatendens over uker.
- Ujevnheter pga. værskifte

In [ ]:
temp_trend = analyse.kolonne_trend("Temperatur", vindu=7)
print("Glidende 7-dagers snitt for temperatur:")
print(temp_trend-head(10))

- Det kan være at de første radene av outputen er manglende verdier. Forklaringen på dette er at vi ikke har nok tidligere datapunkter. 
- Verdiene vi får ut er gjennomsnittet over en 7 dagers periode. 

- Disse er svært yttige i neste steg som er Visualiseringen. 